# Agent Custom

In [1]:
from dotenv import load_dotenv

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

c:\walker\code\langchain-basic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Custom Tool

In [2]:
# 커스텀 TOOL 정의
@tool
def add_function(a: float, b: float) -> float:
    """ Add two numbers"""
    return a + b

In [3]:
prompt = """
너는 수학을 잘하는 전문가야
"""

# 1. LLM 설정
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 2. PROMPT 
prompt = ChatPromptTemplate.from_messages([
    ('system', prompt),
    ('human', '{input}'),
    ('placeholder', '{agent_scratchpad}') # tool 사용 결과를 담는 임시 공간
])

# 3. 단일 AGENT 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=[add_function],
    prompt=prompt
)

# 5. AGENT EXECUTOR 설정
executor = AgentExecutor(
    agent=agent,
    tools=[add_function],
    verbose=True
)

In [4]:
result = executor.invoke({'input' : '2 + 2'})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 2, 'b': 2}`


4.02 + 2는 4입니다.

> Finished chain.
{'input': '2 + 2', 'output': '2 + 2는 4입니다.'}


## 2. Memory + Tool

In [6]:
from typing import Dict 
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

In [7]:
prompt = """
너는 수학을 잘하는 전문가야
"""

# 1. LLM 설정
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 2. PROMPT 
prompt = ChatPromptTemplate.from_messages([
    ('system', prompt),
    MessagesPlaceholder(variable_name='history'), # ('placeholder', '{history}'),
    ('human', '{question}'),
    ('placeholder', '{agent_scratchpad}')
])

# 3. 단일 AGENT 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=[add_function],
    prompt=prompt
)

# 5. AGENT EXECUTOR 설정
executor = AgentExecutor(
    agent=agent,
    tools=[add_function],
    verbose=True
)

In [8]:
stores : Dict[str, InMemoryChatMessageHistory] = {}

def get_stores(session_id: str):
    
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
        
    return stores[session_id]

In [9]:
agent_history = RunnableWithMessageHistory(
    executor, 
    lambda session_id: get_stores(session_id),
    input_messages_key='question',
    history_messages_key='history'
)

In [10]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "5 + 2는?"
    }, config = cfg
)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 5, 'b': 2}`


7.05 + 2는 7입니다.

> Finished chain.


{'question': '5 + 2는?', 'history': [], 'output': '5 + 2는 7입니다.'}

In [ ]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "아까 내가 뭐 물어봤어?"
    }, config = cfg
)



> Entering new AgentExecutor chain...
아까 "5 + 2는?"이라고 물어보셨어요.

> Finished chain.


{'question': '아까 내가 뭐 물어봤어?',
 'history': [HumanMessage(content='5 + 2는?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5 + 2는 7입니다.', additional_kwargs={}, response_metadata={})],
 'output': '아까 "5 + 2는?"이라고 물어보셨어요.'}

In [11]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "방금 뭐 물어봤어?"
    }, config = cfg
)



> Entering new AgentExecutor chain...
방금 "5 + 2는?"이라고 물어보셨어요.

> Finished chain.


{'question': '방금 뭐 물어봤어?',
 'history': [HumanMessage(content='5 + 2는?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5 + 2는 7입니다.', additional_kwargs={}, response_metadata={})],
 'output': '방금 "5 + 2는?"이라고 물어보셨어요.'}

## 3. CSV Tool Custom

In [12]:
from dotenv import load_dotenv

from typing import List, Dict, Optional

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

load_dotenv()

True

In [13]:
import csv

@tool('csv_write', return_direct=False)
def csv_write(csv_path: str, rows: List[Dict], mode: str = 'a') -> str:
    """
    rows(List[dict])를 csv 파일에 저장한다
    - csv_path : csv 경로('result.csv')
    - rows : 딕셔너리 목록, 키가 헤더가 됨, 새파일이면 헤더를 자동 생성
    - mode : 'a' 추가 모드(기본), 'w' 새로 쓰기
    반환 : 작성 행 수/헤더 정보
    """
    
    header = ["date", "query", "title", "url", "snippet"]
    
    with open(csv_path, mode, newline="", encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        for r in rows:
            writer.writerow(r)
    return f'csv 저장 완료 : {csv_path}, 행수 : {len(rows)}, 헤더 : {header}'

## * decorator

In [14]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("함수 실행전")
        result = func(*args, **kwargs)
        print("함수 실행 후")
        return result
    return wrapper

In [15]:
@my_decorator
def hello_custom(name, greeting="반가워요"):
    print(name, "님", greeting)

In [16]:
result = hello_custom("walker")

함수 실행전
walker 님 반가워요
함수 실행 후
